In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

In [20]:
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv'
df = pd.read_csv(url, engine='python')
print(df.head())
print(f'''\nShape : {df.shape}
\nData Types : {df.dtypes} 
\nDtype of first element of Date column : {type(df.iloc[0,0])} ''')

         Date  Temp
0  1981-01-01  20.7
1  1981-01-02  17.9
2  1981-01-03  18.8
3  1981-01-04  14.6
4  1981-01-05  15.8

Shape : (3650, 2)

Data Types : Date     object
Temp    float64
dtype: object 

Dtype of first element of Date column : <class 'str'> 


In [10]:
df2 = df.drop(columns=['Date'])

In [11]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(df2.values)

In [21]:
# Create the training and testing datasets
training_data_len = int(np.ceil(len(scaled_data) * 0.8))
train_data = scaled_data[0:training_data_len, :]
test_data = scaled_data[training_data_len - 60:, :]

def create_dataset(data, time_step=1):
    X, Y = [], []
    for i in range(len(data) - time_step - 1):
        a = data[i:(i + time_step), 0]
        X.append(a)
        Y.append(data[i + time_step, 0])
    return np.array(X), np.array(Y)

time_step = 60
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)


In [22]:
model = Sequential()
model.add(LSTM(units = 50 , return_sequences= True, input_shape=(time_step , 1)))
model.add(LSTM(units=50, return_sequences = False))
model.add(Dense(units=25))
model.add(Dense(units=1))

In [23]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit(X_train, y_train , batch_size = 1, epochs = 20)

Epoch 1/20

2859/2859 [==============================] - 60s 19ms/step - loss: 0.0126
Epoch 2/20
2859/2859 [==============================] - 68s 24ms/step - loss: 0.0101
Epoch 3/20
2859/2859 [==============================] - 97s 34ms/step - loss: 0.0098
Epoch 4/20
1574/2859 [===============>..............] - ETA: 40s - loss: 0.0094

In [ ]:
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

In [ ]:
# Evaluate the model
rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
print(f'Root Mean Squared Error: {rmse}')